In [1]:
import pandas as pd
import numpy as np
import skmem

train_set = pd.read_csv('../data/train.csv')
test_set = pd.read_csv('../data/test.csv')

weather_train_set = pd.read_csv('../data/weather_train.csv')
weather_test_set = pd.read_csv('../data/weather_test.csv')

building_metadata = pd.read_csv('../data/building_metadata.csv')

sample_submission = pd.read_csv('../data/sample_submission.csv')

In [2]:
train_data = pd.merge(train_set, building_metadata, on='building_id')
train_data = pd.merge(train_data, weather_train_set, on=['site_id', 'timestamp'])

In [3]:
mr = skmem.MemReducer()
train_data = mr.fit_transform(train_data)

Getting memory usage.
Memory in: 5092.41 MB
Starting integers.
Downcast 4 standard integer columns.
Starting floats.
auto
Skipping columns that are not np.float64
Downcast 10 float columns.
Starting objects.
Converted 2 columns to categories.
Memory out: 1133.29 MB Reduction: 77.7%



In [4]:
train_data['primary_use'] = pd.Categorical(train_data['primary_use']).codes

In [5]:
test_data = pd.merge(test_set, building_metadata, on='building_id')
test_data = pd.merge(test_data, weather_test_set, on=['site_id', 'timestamp'])

In [6]:
mr = skmem.MemReducer()
test_data = mr.fit_transform(test_data)

Getting memory usage.
Memory in: 10502.28 MB
Starting integers.
Downcast 5 standard integer columns.
Starting floats.
auto
Skipping columns that are not np.float64
Downcast 9 float columns.
Starting objects.
Converted 2 columns to categories.
Memory out: 2336.77 MB Reduction: 77.7%



In [7]:
test_data['primary_use'] = pd.Categorical(test_data['primary_use']).codes

In [8]:
train_data.isnull().sum() / len(train_data) * 100

building_id            0.000000
meter                  0.000000
timestamp              0.000000
meter_reading          0.000000
site_id                0.000000
primary_use            0.000000
square_feet            0.000000
year_built            60.188531
floor_count           82.631315
air_temperature        0.030623
cloud_coverage        43.401776
dew_temperature        0.047924
precip_depth_1_hr     18.178475
sea_level_pressure     5.670259
wind_direction         6.750371
wind_speed             0.264245
dtype: float64

In [9]:
test_data.isnull().sum() / len(test_data) * 100

row_id                 0.000000
building_id            0.000000
meter                  0.000000
timestamp              0.000000
site_id                0.000000
primary_use            0.000000
square_feet            0.000000
year_built            59.148082
floor_count           82.724073
air_temperature        0.055115
cloud_coverage        46.611607
dew_temperature        0.148848
precip_depth_1_hr     18.319990
sea_level_pressure     5.585245
wind_direction         6.698144
wind_speed             0.248346
dtype: float64

In [13]:
train_data.drop(columns=['year_built', 'floor_count'])
print(train_data)

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.000000,0,0,7432,25.0,6.0,20.0,NaN,1019.700012,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.000000,0,0,2720,25.0,6.0,20.0,NaN,1019.700012,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.000000,0,0,5376,25.0,6.0,20.0,NaN,1019.700012,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.000000,0,0,23685,25.0,6.0,20.0,NaN,1019.700012,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.000000,0,0,116607,25.0,6.0,20.0,NaN,1019.700012,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20125600,1400,1,2016-03-24 12:00:00,15.375300,15,4,21168,1.7,NaN,1.7,NaN,1016.799988,0.0,0.0
20125601,1400,1,2016-03-24 13:00:00,25.084801,15,4,21168,2.8,NaN,2.2,NaN,1016.599976,320.0,1.5
20125602,1400,1,2016-03-24 14:00:00,32.343899,15,4,21168,5.6,NaN,4.4,NaN,1015.200012,110.0,2.1
20125603,1400,1,2016-03-24 15:00:00,24.221399,15,4,21168,11.1,NaN,4.4,NaN,1013.900024,150.0,5.1


          building_id  meter            timestamp  meter_reading  site_id  \
0                   0      0  2016-01-01 00:00:00       0.000000        0   
1                   1      0  2016-01-01 00:00:00       0.000000        0   
2                   2      0  2016-01-01 00:00:00       0.000000        0   
3                   3      0  2016-01-01 00:00:00       0.000000        0   
4                   4      0  2016-01-01 00:00:00       0.000000        0   
...               ...    ...                  ...            ...      ...   
20125600         1400      1  2016-03-24 12:00:00      15.375300       15   
20125601         1400      1  2016-03-24 13:00:00      25.084801       15   
20125602         1400      1  2016-03-24 14:00:00      32.343899       15   
20125603         1400      1  2016-03-24 15:00:00      24.221399       15   
20125604         1400      1  2016-03-24 16:00:00      16.355301       15   

          primary_use  square_feet  year_built  floor_count  air_temperatur